## Grupo

227124 Jorge Henrique Monteiro dos Santos

262251 Vitor Mello de Araujo Lima

## Objetivo

Aplicar os vários regressores visto em aula. Busca de hiperparametros.


In [473]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from statistics import mean
import math
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, TimeSeriesSplit, GridSearchCV,RandomizedSearchCV
import numpy as np
from sklearn.model_selection import StratifiedKFold

# Leia

## Leia o arquivo dados4.csv.


In [474]:
url = "https://www.ic.unicamp.br/~wainer/cursos/1s2021/432/dados4.csv"
data = pd.read_csv(url)
data.shape

(690, 15)

In [475]:
data[::]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15
0,1,22.08,11.460,2,k,bb,1.585,0,0,0,1,g,100,1213,0
1,0,22.67,7.000,2,c,bb,0.165,0,0,0,0,g,160,1,0
2,0,29.58,1.750,1,k,bb,1.250,0,0,0,1,g,280,1,0
3,0,21.67,11.500,1,j,j,0.000,1,1,11,1,g,0,1,1
4,1,20.17,8.170,2,aa,bb,1.960,1,1,14,0,g,60,159,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,1,31.57,10.500,2,x,bb,6.500,1,0,0,0,g,0,1,1
686,1,20.67,0.415,2,c,bb,0.125,0,0,0,0,g,0,45,0
687,0,18.83,9.540,2,aa,bb,0.085,1,0,0,0,g,100,1,1
688,0,27.42,14.500,2,x,h,3.085,1,1,1,0,g,120,12,1


##  Converta os atributos categóricos para numéricos (V5,V6,V12)

In [476]:
V5  = pd.get_dummies(data['V5'],  prefix='V5')
V6  = pd.get_dummies(data['V6'],  prefix='V6')
V12 = pd.get_dummies(data['V12'], prefix='V12')

data_num = data.drop(['V5', 'V6','V12' ],  axis=1)
data_num = pd.concat([data_num,V5,V6,V12], axis=1)

data_num.head()

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
0,1,22.08,11.46,2,1.585,0,0,0,1,100,...,0,0,0,0,0,0,0,1,0,0
1,0,22.67,7.00,2,0.165,0,0,0,0,160,...,0,0,0,0,0,0,0,1,0,0
2,0,29.58,1.75,1,1.250,0,0,0,1,280,...,0,0,0,0,0,0,0,1,0,0
3,0,21.67,11.50,1,0.000,1,1,11,1,0,...,0,0,0,1,0,0,0,1,0,0
4,1,20.17,8.17,2,1.960,1,1,14,0,60,...,0,0,0,0,0,0,0,1,0,0


In [477]:
y_data = data_num['V15']
x_data = data_num.drop(['V15'], axis=1)

# Centering and Scaling

In [478]:
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame

x_data_scaled = DataFrame(StandardScaler().fit_transform(x_data), columns=x_data.columns)
x_data_scaled.head()

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
0,0.688737,-0.801052,1.347111,0.542950,-0.190906,-1.047504,-0.864196,-0.493887,1.087908,-0.488358,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.32249,-0.108306,-0.300079
1,-1.451933,-0.751240,0.450548,0.542950,-0.615536,-1.047504,-0.864196,-0.493887,-0.919195,-0.139591,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.32249,-0.108306,-0.300079
2,-1.451933,-0.167856,-0.604823,-1.783978,-0.291083,-1.047504,-0.864196,-0.493887,1.087908,0.557943,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.32249,-0.108306,-0.300079
3,-1.451933,-0.835667,1.355152,-1.783978,-0.664877,0.954650,1.157144,1.769760,1.087908,-1.069637,...,-0.093659,-0.300079,-0.5,9.233093,-0.093659,-0.305782,-0.108306,0.32249,-0.108306,-0.300079
4,0.688737,-0.962306,0.685745,0.542950,-0.078768,0.954650,1.157144,2.387119,-0.919195,-0.720870,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.32249,-0.108306,-0.300079


In [479]:
x_data_scaled

,V1,V2,V3,V4,V7,V8,V9,V10,V11,V13,...,V6_dd,V6_ff,V6_h,V6_j,V6_o,V6_v,V6_z,V12_g,V12_p,V12_s
0,0.688737,-0.801052,1.347111,0.542950,-0.190906,-1.047504,-0.864196,-0.493887,1.087908,-0.488358,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
1,-1.451933,-0.751240,0.450548,0.542950,-0.615536,-1.047504,-0.864196,-0.493887,-0.919195,-0.139591,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
2,-1.451933,-0.167856,-0.604823,-1.783978,-0.291083,-1.047504,-0.864196,-0.493887,1.087908,0.557943,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
3,-1.451933,-0.835667,1.355152,-1.783978,-0.664877,0.954650,1.157144,1.769760,1.087908,-1.069637,...,-0.093659,-0.300079,-0.5,9.233093,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
4,0.688737,-0.962306,0.685745,0.542950,-0.078768,0.954650,1.157144,2.387119,-0.919195,-0.720870,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,0.688737,0.000152,1.154129,0.542950,1.278852,0.954650,-0.864196,-0.493887,-0.919195,-1.069637,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
686,0.688737,-0.920093,-0.873189,0.542950,-0.627497,-1.047504,-0.864196,-0.493887,-0.919195,-1.069637,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
687,-1.451933,-1.075437,0.961146,0.542950,-0.639459,0.954650,-0.864196,-0.493887,-0.919195,-0.488358,...,-0.093659,-0.300079,-0.5,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079
688,-1.451933,-0.350217,1.958221,0.542950,0.257647,0.954650,1.157144,-0.288101,-0.919195,-0.372103,...,-0.093659,-0.300079,2.0,-0.108306,-0.093659,-0.305782,-0.108306,0.322490,-0.108306,-0.300079


In [480]:
assert all (x_data_scaled.mean().values > -10e-14)
assert all (x_data_scaled.mean().values <  10e-14)
assert all (x_data_scaled.std() < 1.001)
assert all (x_data_scaled.std() > 0.999)


# Classificação 

In [481]:
# Generate all classifiers and parameters
import pprint
from sklearn.linear_model import LogisticRegression
import scipy.stats as stats
from sklearn.utils.fixes import loguniform
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
import random
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB


test_model = {

    'Logistic Regression': {
        'regressor' : LogisticRegression,
        'parameters': {'max_iter': [2000]},
        'n_iter'    : 1,
        'results'   : {}
    },
    'Logistic Regression - L2': {
        'regressor': LogisticRegression,
        'parameters': {
            'penalty': ['l2'],
            'C' : loguniform(10**-3, 10**3),
            'max_iter': [2000]
        },
        'n_iter' : 10,
        'results' : {}
    },
    'Linear Discriminant Analysisr':{
        'regressor': LinearDiscriminantAnalysis,
        'parameters': {},
        'n_iter'    : 1,
        'results'   : {}
    },
    'Quadratic Discriminant Analysis':{
        'regressor': QuadraticDiscriminantAnalysis,
        'parameters': {},
        'n_iter'    : 1,
        'results'   : {}
    },
    'K-Nearest Neighbors': {
        'regressor': KNeighborsClassifier,
        'parameters': {
            'n_neighbors': stats.randint(1,302)
            },
        'n_iter' : 10,
        'results' : {} 
    },
    'SVM Linear': {
        'regressor': SVC,
        'parameters': {
            'C'      : loguniform(2**-5, 2**15),
            'kernel' : ['linear'],
            'probability': [True]
        },
        'n_iter' : 10,
        'results' : {} 
    },
    'SVM RBF': {
        'regressor': SVC,
        'parameters': {
            'C'      : loguniform(2**-5, 2**15),
            'gamma'  : loguniform(2**-9, 2**3),
            'kernel' : ['rbf'],
            'probability': [True]
        },
        'n_iter' : 10,
        'results' : {} 
    },
    'Multi-layer Perceptron': {
        'regressor': MLPClassifier,
        'parameters': {
            'hidden_layer_sizes': np.arange(5, 20+1, 3),
            'max_iter': [5000]
            },
        'n_iter' : 10,
        'results' : {} 
    },
    'Decision Tree': {
        'regressor': DecisionTreeClassifier,
        'parameters': {
            'ccp_alpha': uniform(loc=0.0, scale=0.04)
            },
        'n_iter' : 10,
        'results' : {} 
    },
    'Random Forest': {
        'regressor': RandomForestClassifier,
        'parameters': {
            'n_estimators': [10, 100, 1000],
            'max_features': [5,8,10]
            },
        'n_iter' : 10,
        'results' : {} 
    },
    'Gradient Boost Machine': {
        'regressor': GradientBoostingClassifier,
        'parameters': {
            'n_estimators': np.arange(5, 101, 1),
            'learning_rate': uniform(loc=0.01, scale=(0.3-0.01)),
            'max_depth': [2,3]
            },
        'n_iter' : 10,
        'results' : {} 
    },
    'Gaussian Naive Bayes': {
        'regressor': GaussianNB,
        'parameters': {},
        'n_iter'    : 1,
        'results'   : {} 
    }
}

## Nested-Cross Validation

- Outer-loop: 4 repetições de split de 70% para treino e 30% para teste no outer loop
- Inner-loop: 3-fold estratificado

A implemetação utilizada neste trabalho é baseada no seguinte exemplo:

https://machinelearningmastery.com/nested-cross-validation-for-machine-learning-with-python/

In [482]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from numpy import mean
from numpy import std

def nested_cross_val_random(model,parameters,X_data, y_data, n_it, classifier):
    auc_list = []
    
    for i in range(4):
        # Create test and train sets
        X_train,X_test,y_train,y_test=train_test_split(X_data, y_data, test_size=0.3)

        inner_cv = StratifiedKFold(n_splits=3)

        rnd_search = RandomizedSearchCV(estimator=model(), param_distributions=parameters, cv=inner_cv, scoring='roc_auc', refit=True, n_iter=n_it, n_jobs=12)
        result = rnd_search.fit(X_train, y_train)
        # get the best performing model fit on the whole training set
        best_model = result.best_estimator_
        auc = roc_auc_score(y_test,best_model.predict_proba(X_test)[:, 1])
        auc_list.append(auc)
    
                        
    return mean(auc_list), std(auc_list)

    
for classifier in test_model.keys():
    print(classifier)
    auc_mean,auc_std = nested_cross_val_random(test_model[classifier]['regressor'],
                                               test_model[classifier]['parameters'],
                                               x_data_scaled,y_data, 
                                               test_model[classifier]['n_iter'],
                                               classifier)
    test_model[classifier]['results']['auc_mean'] = auc_mean
    test_model[classifier]['results']['auc_std']  = auc_std

Logistic Regression
Logistic Regression - L2
Linear Discriminant Analysisr
Quadratic Discriminant Analysis
K-Nearest Neighbors
/home/vmello/miniconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/vmello/miniconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/vmello/miniconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/home/vmello/miniconda3/lib/python3.9/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
SVM Linear
SVM RBF
Multi-layer Perceptron
/home/vmello/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 6 is smaller tha

In [483]:
import pprint
cls_results_dict = {}

for classifier in test_model.keys():
    cls_results_dict[classifier] = {'auc_men':test_model[classifier]['results']['auc_mean'],
                                    'auc_std':test_model[classifier]['results']['auc_std']}

pprint.pprint(cls_results_dict)



{'Decision Tree': {'auc_men': 0.9088409742793432,
                   'auc_std': 0.01709131051122639},
 'Gaussian Naive Bayes': {'auc_men': 0.8505942986776118,
                          'auc_std': 0.026975802910637198},
 'Gradient Boost Machine': {'auc_men': 0.9444885290254885,
                            'auc_std': 0.016874891125064063},
 'K-Nearest Neighbors': {'auc_men': 0.9127618257424943,
                         'auc_std': 0.01900042301153708},
 'Linear Discriminant Analysisr': {'auc_men': 0.9053055321414869,
                                   'auc_std': 0.007788908513463214},
 'Logistic Regression': {'auc_men': 0.9251128867679435,
                         'auc_std': 0.019969398306604703},
 'Logistic Regression - L2': {'auc_men': 0.9236849156685958,
                              'auc_std': 0.012218430508017365},
 'Multi-layer Perceptron': {'auc_men': 0.9057846302093933,
                            'auc_std': 0.008945351800091835},
 'Quadratic Discriminant Analysis': {'auc_men': 0.

# Tabela Final

Segue a tabela com o valor de _AUC_ médio e seu desvio padrão calculado no _outer loop_ para cada algoritmo:

In [484]:
cls_table = df.from_dict(cls_results_dict, orient='index')
cls_table

,auc_men,auc_std
Logistic Regression,0.925113,0.019969
Logistic Regression - L2,0.923685,0.012218
Linear Discriminant Analysisr,0.905306,0.007789
Quadratic Discriminant Analysis,0.809922,0.038590
K-Nearest Neighbors,0.912762,0.019000
SVM Linear,0.928402,0.016458
SVM RBF,0.900253,0.015831
Multi-layer Perceptron,0.905785,0.008945
Decision Tree,0.908841,0.017091
Random Forest,0.936944,0.016106
